In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import spacy
import matplotlib as mpl
import seaborn as sns
import warnings


warnings.filterwarnings('ignore')
plt.style.use('dark_background')
mpl.rcParams['axes.prop_cycle'] = plt.cycler(color=['blue'])

In [90]:
#get encoding of the data:
import chardet

with open('./data/train.csv' , 'rb') as f:
    result = chardet.detect(f.read(100000))
    # print(result)

df = pd.read_csv('./data/train.csv' , encoding = result['encoding'])
df.columns = ['itemid' , 'sentiment' , 'text']
#lower the tweets:
df['text'] = df['text'].str.lower()
df.head()

,itemid,sentiment,text
0,1,0,"@railminindia my pnr is 8348062961, i am in wa..."
1,2,0,@sureshpprabhu @railminindia ac not working in...
2,3,0,@railminindia i'm traveling to chennai by trai...
3,4,5,@railminindia irctc is not responding at the t...
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1366 entries, 0 to 1365
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   itemid     1366 non-null   int64 
 1   sentiment  1366 non-null   int64 
 2   text       1366 non-null   object
dtypes: int64(2), object(1)
memory usage: 32.1+ KB


## Clearning and feature engineering:


In [92]:
df.head()

,itemid,sentiment,text
0,1,0,"@railminindia my pnr is 8348062961, i am in wa..."
1,2,0,@sureshpprabhu @railminindia ac not working in...
2,3,0,@railminindia i'm traveling to chennai by trai...
3,4,5,@railminindia irctc is not responding at the t...
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...


### punctutation removal and lower-text


In [93]:
import string

transtable = str.maketrans("" , "" , string.punctuation)
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### User mention extration:

create a column that has a list of all the @handles mentioned in the text.


In [94]:
import regex as re
samplet = df['text'].iloc[4]
pattern = r'@\w+'
print(f"{samplet=}")
np.unique(re.findall(pattern , samplet))

samplet='@drmbhopal @railminindia @sanjaygupta2012 @drmncrald matter notified to concerned official @bhusavaldivn'


array(['@bhusavaldivn', '@drmbhopal', '@drmncrald', '@railminindia',
       '@sanjaygupta2012'], dtype='<U16')

In [95]:
df['mentions'] = df['text'].apply(lambda x : re.findall(pattern , x))
df.head()

,itemid,sentiment,text,mentions
0,1,0,"@railminindia my pnr is 8348062961, i am in wa...",[@railminindia]
1,2,0,@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia]"
2,3,0,@railminindia i'm traveling to chennai by trai...,[@railminindia]
3,4,5,@railminindia irctc is not responding at the t...,[@railminindia]
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ..."


### hashtag extraction:

- Lowered text for hastags


In [96]:
#regex pattern with negative look-behind:
hashpattern = r"(?<!\w)#\w+"
samples = df['text'].iloc[:500]

for i , tweet in enumerate(samples):
    if(re.search(hashpattern , tweet)):
        print(f"{i=} , {tweet=} , {re.findall(hashpattern , tweet)}")

i=8 , tweet='enter to win $150 amazon gift card! #books #pnr #urbanfantasy #romance https://t.co/jbfxzde7p2' , ['#books', '#pnr', '#urbanfantasy', '#romance']
i=11 , tweet='@railminindia #swatch bharat abhiyaan# train stops at kozhikod(calicut) but no one comes in 12218 coacha1 #no water# https://t.co/h5uudjuvel' , ['#swatch', '#no']
i=54 , tweet='@rpfcrsur @railminindia how many months or should i say years should i wait for the conscience of #corrupt #rotten‚ä¶ https://t.co/2h0v7kwzgy' , ['#corrupt', '#rotten']
i=71 , tweet='rt @karailway: provide 1min stoppage for karnataka sampark kranthi at #haveri @drmmys @gmswr @railminindia @piyushgoyaloffc‚ä¶ ' , ['#haveri']
i=103 , tweet='@railminindia got double tatkal tickets cnfrmd through same login costing 12k technical issue costed double[irctc #3515189] @drmsecunderabad' , ['#3515189']
i=108 , tweet='@sureshpprabhu @railminindia @gmner_gkp @drm_asn train 15047  light are not working passengers are suffering #help https://t.co/6fnijv8vg

In [97]:
df['hashtags'] = df['text'].apply(lambda x : re.findall(hashpattern , x))
df.iloc[:20]

,itemid,sentiment,text,mentions,hashtags
0,1,0,"@railminindia my pnr is 8348062961, i am in wa...",[@railminindia],[]
1,2,0,@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia]",[]
2,3,0,@railminindia i'm traveling to chennai by trai...,[@railminindia],[]
3,4,5,@railminindia irctc is not responding at the t...,[@railminindia],[]
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",[]
5,6,6,@railminindia if you can't give justice to gra...,[@railminindia],[]
6,7,4,@sureshpprabhu @railminindiaplz wrkout smthng ...,"[@sureshpprabhu, @railminindiaplz]",[]
7,8,0,@railminindia @mumbairailusers dirty water flo...,"[@railminindia, @mumbairailusers]",[]
8,9,6,enter to win $150 amazon gift card! #books #pn...,[],"[#books, #pnr, #urbanfantasy, #romance]"
9,10,3,@railminindia still vendors are selling local ...,[@railminindia],[]


### Extract urls:


In [98]:
urlpattern = r"https?://\S+|www\.|S+"
for i , tweet in enumerate(samples):
    if(re.search(hashpattern , tweet)):
        print(f"{i=} , {tweet=} , {re.findall(hashpattern , tweet)}")

i=8 , tweet='enter to win $150 amazon gift card! #books #pnr #urbanfantasy #romance https://t.co/jbfxzde7p2' , ['#books', '#pnr', '#urbanfantasy', '#romance']
i=11 , tweet='@railminindia #swatch bharat abhiyaan# train stops at kozhikod(calicut) but no one comes in 12218 coacha1 #no water# https://t.co/h5uudjuvel' , ['#swatch', '#no']
i=54 , tweet='@rpfcrsur @railminindia how many months or should i say years should i wait for the conscience of #corrupt #rotten‚ä¶ https://t.co/2h0v7kwzgy' , ['#corrupt', '#rotten']
i=71 , tweet='rt @karailway: provide 1min stoppage for karnataka sampark kranthi at #haveri @drmmys @gmswr @railminindia @piyushgoyaloffc‚ä¶ ' , ['#haveri']
i=103 , tweet='@railminindia got double tatkal tickets cnfrmd through same login costing 12k technical issue costed double[irctc #3515189] @drmsecunderabad' , ['#3515189']
i=108 , tweet='@sureshpprabhu @railminindia @gmner_gkp @drm_asn train 15047  light are not working passengers are suffering #help https://t.co/6fnijv8vg

In [100]:
def processUrl(text):
    return pd.Series((re.findall(urlpattern , text) , re.sub(urlpattern , "" , text)))
df[['urls' ,'text']] = df['text'].apply(processUrl)
df.head(20)

,itemid,sentiment,text,mentions,hashtags,urls
0,1,0,"@railminindia my pnr is 8348062961, i am in wa...",[@railminindia],[],[]
1,2,0,@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia]",[],[]
2,3,0,@railminindia i'm traveling to chennai by trai...,[@railminindia],[],[]
3,4,5,@railminindia irctc is not responding at the t...,[@railminindia],[],[]
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",[],[]
5,6,6,@railminindia if you can't give justice to gra...,[@railminindia],[],[]
6,7,4,@sureshpprabhu @railminindiaplz wrkout smthng ...,"[@sureshpprabhu, @railminindiaplz]",[],[]
7,8,0,@railminindia @mumbairailusers dirty water flo...,"[@railminindia, @mumbairailusers]",[],[https://t.co/mbzuofklxq]
8,9,6,enter to win $150 amazon gift card! #books #pn...,[],"[#books, #pnr, #urbanfantasy, #romance]",[https://t.co/jbfxzde7p2]
9,10,3,@railminindia still vendors are selling local ...,[@railminindia],[],[https://t.co/wdfyllmmek]


In [2]:
emojit = "Hey xoxoxoxoxo 💪🙈😘"

In [4]:
import emoji

emoji.demojize(emojit)

'Hey xoxoxoxoxo :flexed_biceps::see-no-evil_monkey::face_blowing_a_kiss:'

### Translate emojis to text:


In [107]:
df['etext'] = df['text'].apply(lambda x : emoji.demojize(x))
df.head(20)

,itemid,sentiment,text,mentions,hashtags,urls,etext
0,1,0,"@railminindia my pnr is 8348062961, i am in wa...",[@railminindia],[],[],"@railminindia my pnr is 8348062961, i am in wa..."
1,2,0,@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia]",[],[],@sureshpprabhu @railminindia ac not working in...
2,3,0,@railminindia i'm traveling to chennai by trai...,[@railminindia],[],[],@railminindia i'm traveling to chennai by trai...
3,4,5,@railminindia irctc is not responding at the t...,[@railminindia],[],[],@railminindia irctc is not responding at the t...
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",[],[],@drmbhopal @railminindia @sanjaygupta2012 @drm...
5,6,6,@railminindia if you can't give justice to gra...,[@railminindia],[],[],@railminindia if you can't give justice to gra...
6,7,4,@sureshpprabhu @railminindiaplz wrkout smthng ...,"[@sureshpprabhu, @railminindiaplz]",[],[],@sureshpprabhu @railminindiaplz wrkout smthng ...
7,8,0,@railminindia @mumbairailusers dirty water flo...,"[@railminindia, @mumbairailusers]",[],[https://t.co/mbzuofklxq],@railminindia @mumbairailusers dirty water flo...
8,9,6,enter to win $150 amazon gift card! #books #pn...,[],"[#books, #pnr, #urbanfantasy, #romance]",[https://t.co/jbfxzde7p2],enter to win $150 amazon gift card! #books #pn...
9,10,3,@railminindia still vendors are selling local ...,[@railminindia],[],[https://t.co/wdfyllmmek],@railminindia still vendors are selling local ...


## Tokenization and comparision b/w tokenization approaches for nltk and spacy:


In [141]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 6.8 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [145]:
from nltk.tokenize import word_tokenize , sent_tokenize
import spacy
nlp = spacy.load("en_core_web_sm")
nlpl = spacy.load("en_core_web_lg")

# get 100 random tweets
samples = df['etext'].sample(100 , replace= False)
samples


388    fan isnt working for the last 6 hours at the t...
649    @railminindia unreserved people sitting near t...
170    @sureshpprabhu @railminindia @rajeshbhargav19 ...
329    @railminindia sir,???? ?????? ???? tweet delet...
659    @railminindia dadar-bikaner express(12490) run...
                             ...                        
259    @railminindia @sureshpprabhu 12904 brc to bct....
706    @railminindia @sureshpprabhu : mobile charging...
137    scenic #indianrailways near #igatpuri on #mumb...
554    pnr 4512791357 fan chocked, repairmen came bt ...
323    @sureshpprabhu @railminindia @westernrly train...
Name: etext, Length: 100, dtype: object

In [146]:
# number of unique words for nltk and spacy:
nltkavg = 0
lwords = []
for tweet in samples:
    words = [word for word in word_tokenize(tweet)]
    nltkavg += len(words)
    lwords+= [*words]
nltkavg /= 100
print(f"{len(np.unique(lwords))=} , {nltkavg=}")


len(np.unique(lwords))=848 , nltkavg=21.89


In [147]:
# token analysis using spacy:
savg = 0
swords = []

for tweet in samples:
    words = [token.text for token in nlpl(tweet)]
    savg += len(words)
    swords += [*words]

savg /= 100
print(f"{len(np.unique(swords))=} , {savg=}")

len(np.unique(swords))=850 , savg=20.15


### Stem text using potter stemmer


In [158]:
%%timeit
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

def stemtweet(tweet):
    #tokenize tweet:
    words = [token.text for token in nlp(tweet)]
    #stem the tokens:
    swords = [stemmer.stem(word) for word in words]
    #return stemmed tweet:
    return [*swords]
# for tweet in samples:
#     print(f"{tweet= } \n, {stemtweet(tweet)}")

def lemmatizetweet(tweet):
    lwords = [word.lemma_ for word in nlp(tweet) if not word.is_stop]
    return " ".join(lwords)

def getpos(tweet):
    tags = [word.tag_ for word in nlp(tweet) if not word.is_stop]
    return tags

df['stemmed_text'] = df['text'].apply(lambda x : stemtweet(x))
df['lemmatized_text'] = df['text'].apply(lambda x : lemmatizetweet(x))
df['pos_tags'] = df['text'].apply(lambda x : getpos(x))
# df.head()

16.3 s ± 219 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [159]:
df.head()

,itemid,sentiment,text,mentions,hashtags,urls,etext,stemmed_text,lemmatized_text,pos_tags,entities
0,1,0,"@railminindia my pnr is 8348062961, i am in wa...",[@railminindia],[],[],"@railminindia my pnr is 8348062961, i am in wa...","[@railminindia, my, pnr, is, 8348062961, ,, i,...","@railminindia pnr 8348062961 , way water toile...","[NNS, NNS, CD, ,, NN, NN, NN, NN, NN, ., VB, J...",[ORG]
1,2,0,@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia]",[],[],@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia, ac, not, work,...",@sureshpprabhu @railminindia ac work prayag lu...,"[NNP, NNP, NNP, VBG, NNP, NN, NN, NN, VBD, NN,...","[DATE, TIME, ORG]"
2,3,0,@railminindia i'm traveling to chennai by trai...,[@railminindia],[],[],@railminindia i'm traveling to chennai by trai...,"[@railminindia, i, 'm, travel, to, chennai, by...",@railminindia travel chennai train 16102 coach...,"[NN, VBG, VB, VB, CD, NN, NNP, HYPH, CD, ., NN...",[CARDINAL]
3,4,5,@railminindia irctc is not responding at the t...,[@railminindia],[],[],@railminindia irctc is not responding at the t...,"[@railminindia, irctc, is, not, respond, at, t...",@railminindia irctc respond time tatkal booking .,"[NN, NN, VBG, NN, NN, NN, .]",[ORG]
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",[],[],@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[NNP, NNP, NNP, NNP, NN, VBD, JJ, JJ, NNP]",[]


## Named Entity Recognizition and Domain Mapping:


In [155]:
for tweet in samples:
    stweet = nlpl(tweet)
    print(f"{tweet=}")
    for ent in stweet.ents:
        print(f"{ent.text=} , {ent.label_}")
    print("*"*400)

tweet='fan isnt working for the last 6 hours at the time of peak summer, do help  pnr 4512791357 '
ent.text='the last 6 hours' , TIME
ent.text='4512791357' , CARDINAL
****************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
tweet='@railminindia unreserved people sitting near toilets female passengers finding difficult to use washroom pnr 8647770808 coach s12'
ent.text='@railminindia' , ORG
ent.text='8647770808' , PRODUCT
***************************************************************************************************************************************************************************************************************************************

In [154]:
def getEntities(tweet):
    elist = set()
    for ent in nlpl(tweet).ents:
        elist.add(ent.label_)
    return [*elist]

df['entities'] = df['text'].apply(getEntities)
df.head()

,itemid,sentiment,text,mentions,hashtags,urls,etext,stemmed_text,lemmatized_text,pos_tags,entities
0,1,0,"@railminindia my pnr is 8348062961, i am in wa...",[@railminindia],[],[],"@railminindia my pnr is 8348062961, i am in wa...","[@railminindia, my, pnr, is, 8348062961, ,, i,...","[@railminindia, pnr, 8348062961, ,, way, water...","[NNS, NNS, CD, ,, NN, NN, NN, NN, NN, ., VB, J...",[ORG]
1,2,0,@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia]",[],[],@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia, ac, not, work,...","[@sureshpprabhu, @railminindia, ac, work, pray...","[NNP, NNP, NNP, VBG, NNP, NN, NN, NN, VBD, NN,...","[DATE, TIME, ORG]"
2,3,0,@railminindia i'm traveling to chennai by trai...,[@railminindia],[],[],@railminindia i'm traveling to chennai by trai...,"[@railminindia, i, 'm, travel, to, chennai, by...","[@railminindia, travel, chennai, train, 16102,...","[NN, VBG, VB, VB, CD, NN, NNP, HYPH, CD, ., NN...",[CARDINAL]
3,4,5,@railminindia irctc is not responding at the t...,[@railminindia],[],[],@railminindia irctc is not responding at the t...,"[@railminindia, irctc, is, not, respond, at, t...","[@railminindia, irctc, respond, time, tatkal, ...","[NN, NN, VBG, NN, NN, NN, .]",[ORG]
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",[],[],@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...","[@drmbhopal, @railminindia, @sanjaygupta2012, ...","[NNP, NNP, NNP, NNP, NN, VBD, JJ, JJ, NNP]",[]


## Sentiment and Emotion Analysis:


### Vader scores


In [160]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/sanketmishra/nltk_data...


True

In [163]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

for tweet in samples:
    print(f"{sia.polarity_scores(tweet)}")

{'neg': 0.0, 'neu': 0.75, 'pos': 0.25, 'compound': 0.6124}
{'neg': 0.135, 'neu': 0.865, 'pos': 0.0, 'compound': -0.3612}
{'neg': 0.136, 'neu': 0.864, 'pos': 0.0, 'compound': -0.296}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.196, 'neu': 0.804, 'pos': 0.0, 'compound': -0.5267}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.0, 'neu': 0.57, 'pos': 0.43, 'compound': 0.807}
{'neg': 0.367, 'neu': 0.513, 'pos': 0.12, 'compound': -0.6996}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.339, 'neu': 0.661, 'pos': 0.0, 'compound': -0.6249}
{'neg': 0.0, 'neu': 0.813, 'pos': 0.187, 'compound': 0.3182}
{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
{'neg': 0.313, 'neu': 0.687, 'pos': 0.0, 'compound': -0.7479}
{'neg': 0.124, 'neu': 0.876, 'pos': 0.0, 'compound': -0.5256}
{'neg': 0.227, 'neu': 0.773, 'pos': 0.0, 'compound': -0.6249}
{'neg': 0.102, 'neu': 0.777, 'pos': 0.121, 'compound': 0

In [167]:
df['vader'] = df['text'].apply(lambda x : sia.polarity_scores(x)['compound'])
df.head()

,itemid,sentiment,text,mentions,hashtags,urls,etext,stemmed_text,lemmatized_text,pos_tags,entities,vader
0,1,0,"@railminindia my pnr is 8348062961, i am in wa...",[@railminindia],[],[],"@railminindia my pnr is 8348062961, i am in wa...","[@railminindia, my, pnr, is, 8348062961, ,, i,...","@railminindia pnr 8348062961 , way water toile...","[NNS, NNS, CD, ,, NN, NN, NN, NN, NN, ., VB, J...",[ORG],-0.4767
1,2,0,@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia]",[],[],@sureshpprabhu @railminindia ac not working in...,"[@sureshpprabhu, @railminindia, ac, not, work,...",@sureshpprabhu @railminindia ac work prayag lu...,"[NNP, NNP, NNP, VBG, NNP, NN, NN, NN, VBD, NN,...","[DATE, TIME, ORG]",-0.2235
2,3,0,@railminindia i'm traveling to chennai by trai...,[@railminindia],[],[],@railminindia i'm traveling to chennai by trai...,"[@railminindia, i, 'm, travel, to, chennai, by...",@railminindia travel chennai train 16102 coach...,"[NN, VBG, VB, VB, CD, NN, NNP, HYPH, CD, ., NN...",[CARDINAL],0.0000
3,4,5,@railminindia irctc is not responding at the t...,[@railminindia],[],[],@railminindia irctc is not responding at the t...,"[@railminindia, irctc, is, not, respond, at, t...",@railminindia irctc respond time tatkal booking .,"[NN, NN, VBG, NN, NN, NN, .]",[ORG],0.0000
4,5,7,@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",[],[],@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[@drmbhopal, @railminindia, @sanjaygupta2012, ...",@drmbhopal @railminindia @sanjaygupta2012 @drm...,"[NNP, NNP, NNP, NNP, NN, VBD, JJ, JJ, NNP]",[],0.0258
